# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 25 2022 2:53PM,238103,10,MET210467,"Methapharm, Inc.",Released
1,May 25 2022 2:53PM,238103,10,MET210495,"Methapharm, Inc.",Released
2,May 25 2022 2:53PM,238103,10,MET210496,"Methapharm, Inc.",Released
3,May 25 2022 2:53PM,238103,10,MET210599,"Methapharm, Inc.",Released
4,May 25 2022 2:53PM,238103,10,MET210600,"Methapharm, Inc.",Released
5,May 25 2022 2:53PM,238103,10,MET210601,"Methapharm, Inc.",Released
6,May 25 2022 2:53PM,238103,10,MET210602,"Methapharm, Inc.",Released
7,May 25 2022 2:53PM,238103,10,MET210603,"Methapharm, Inc.",Released
8,May 25 2022 2:53PM,238103,10,MET210604,"Methapharm, Inc.",Released
9,May 25 2022 2:53PM,238103,10,MET210605,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,238099,Released,4
33,238100,Released,1
34,238101,Released,20
35,238102,Executing,1
36,238103,Released,13


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238099,NaN,NaN,4.0
238100,NaN,NaN,1.0
238101,NaN,NaN,20.0
238102,NaN,1.0,NaN
238103,NaN,NaN,13.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238021,0.0,28.0,7.0
238033,0.0,0.0,2.0
238037,19.0,1.0,0.0
238042,0.0,1.0,0.0
238045,6.0,16.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238021,0,28,7
238033,0,0,2
238037,19,1,0
238042,0,1,0
238045,6,16,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238021,0,28,7
1,238033,0,0,2
2,238037,19,1,0
3,238042,0,1,0
4,238045,6,16,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238021,,28,7
1,238033,,,2
2,238037,19,1,
3,238042,,1,
4,238045,6,16,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 25 2022 2:53PM,238103,10,"Methapharm, Inc."
13,May 25 2022 2:51PM,238102,22,"NBTY Global, Inc."
14,May 25 2022 2:49PM,238101,20,Reliable 1 Laboratories LLC
34,May 25 2022 2:45PM,238100,22,"NBTY Global, Inc."
35,May 25 2022 2:39PM,238099,10,"Methapharm, Inc."
39,May 25 2022 2:31PM,238094,20,Alumier Labs Inc.
79,May 25 2022 2:25PM,238095,10,ISDIN Corporation
107,May 25 2022 2:25PM,238097,10,ISDIN Corporation
108,May 25 2022 2:25PM,238096,18,Hush Hush
112,May 25 2022 2:01PM,238092,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 25 2022 2:53PM,238103,10,"Methapharm, Inc.",,,13
1,May 25 2022 2:51PM,238102,22,"NBTY Global, Inc.",,1,
2,May 25 2022 2:49PM,238101,20,Reliable 1 Laboratories LLC,,,20
3,May 25 2022 2:45PM,238100,22,"NBTY Global, Inc.",,,1
4,May 25 2022 2:39PM,238099,10,"Methapharm, Inc.",,,4
5,May 25 2022 2:31PM,238094,20,Alumier Labs Inc.,,,40
6,May 25 2022 2:25PM,238095,10,ISDIN Corporation,,,28
7,May 25 2022 2:25PM,238097,10,ISDIN Corporation,,,1
8,May 25 2022 2:25PM,238096,18,Hush Hush,,,4
9,May 25 2022 2:01PM,238092,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 2:53PM,238103,10,"Methapharm, Inc.",13,,
1,May 25 2022 2:51PM,238102,22,"NBTY Global, Inc.",,1,
2,May 25 2022 2:49PM,238101,20,Reliable 1 Laboratories LLC,20,,
3,May 25 2022 2:45PM,238100,22,"NBTY Global, Inc.",1,,
4,May 25 2022 2:39PM,238099,10,"Methapharm, Inc.",4,,
5,May 25 2022 2:31PM,238094,20,Alumier Labs Inc.,40,,
6,May 25 2022 2:25PM,238095,10,ISDIN Corporation,28,,
7,May 25 2022 2:25PM,238097,10,ISDIN Corporation,1,,
8,May 25 2022 2:25PM,238096,18,Hush Hush,4,,
9,May 25 2022 2:01PM,238092,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 2:53PM,238103,10,"Methapharm, Inc.",13,,
1,May 25 2022 2:51PM,238102,22,"NBTY Global, Inc.",,1,
2,May 25 2022 2:49PM,238101,20,Reliable 1 Laboratories LLC,20,,
3,May 25 2022 2:45PM,238100,22,"NBTY Global, Inc.",1,,
4,May 25 2022 2:39PM,238099,10,"Methapharm, Inc.",4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 2:53PM,238103,10,"Methapharm, Inc.",13.0,NaN,NaN
1,May 25 2022 2:51PM,238102,22,"NBTY Global, Inc.",NaN,1.0,NaN
2,May 25 2022 2:49PM,238101,20,Reliable 1 Laboratories LLC,20.0,NaN,NaN
3,May 25 2022 2:45PM,238100,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,May 25 2022 2:39PM,238099,10,"Methapharm, Inc.",4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 2:53PM,238103,10,"Methapharm, Inc.",13.0,0.0,0.0
1,May 25 2022 2:51PM,238102,22,"NBTY Global, Inc.",0.0,1.0,0.0
2,May 25 2022 2:49PM,238101,20,Reliable 1 Laboratories LLC,20.0,0.0,0.0
3,May 25 2022 2:45PM,238100,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,May 25 2022 2:39PM,238099,10,"Methapharm, Inc.",4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3333094,79.0,22.0,25.0
16,238079,1.0,0.0,0.0
18,476129,6.0,0.0,0.0
19,714197,1.0,2.0,0.0
20,1190384,101.0,30.0,0.0
22,1190424,4.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3333094,79.0,22.0,25.0
1,16,238079,1.0,0.0,0.0
2,18,476129,6.0,0.0,0.0
3,19,714197,1.0,2.0,0.0
4,20,1190384,101.0,30.0,0.0
5,22,1190424,4.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,79.0,22.0,25.0
1,16,1.0,0.0,0.0
2,18,6.0,0.0,0.0
3,19,1.0,2.0,0.0
4,20,101.0,30.0,0.0
5,22,4.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,79.0
1,16,Released,1.0
2,18,Released,6.0
3,19,Released,1.0
4,20,Released,101.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20,22
Status,,,,,,
Completed,25.0,0.0,0.0,0.0,0.0,0.0
Executing,22.0,0.0,0.0,2.0,30.0,1.0
Released,79.0,1.0,6.0,1.0,101.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20,22
0,Completed,25.0,0.0,0.0,0.0,0.0,0.0
1,Executing,22.0,0.0,0.0,2.0,30.0,1.0
2,Released,79.0,1.0,6.0,1.0,101.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20,22
0,Completed,25.0,0.0,0.0,0.0,0.0,0.0
1,Executing,22.0,0.0,0.0,2.0,30.0,1.0
2,Released,79.0,1.0,6.0,1.0,101.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()